In [1]:
import ee
import datetime
import os
import itertools
import sys
import re

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('..')

In [4]:
from BackGround_modules.Class_2_Classify_Fourier_Img   import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

##### Prepare basic parameters

In [5]:
# define the years to be classified
year_name  = '2017_2019'
year_start = '2019-01-01'
year_end   = '2019-12-31'

# import north_china_plain boundary
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

In [6]:
year_name

'2017_2019'

##### Prepare input_image for classification

In [7]:
# prepare the climate data
Climate_type = ['lrad','prec','pres','shum','srad','temp','wind']
Climate_img = [ee.Image(f"users/wangjinzhulala/North_China_Plain_Python/Climate_data/{i}/{i}_2018").rename(i) for i in Climate_type]

In [8]:
# Function to mask clouds using the Sentinel-2 QA band.
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask  = 1 << 10;
    cirrusBitMask = 1 << 11;

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask)\
                 .select("B.*")\
                 .copyProperties(image, ["system:time_start"])


In [9]:
# import Sentinel img
Sentinel_img =  ee.ImageCollection("COPERNICUS/S2_SR")\
                  .filterBounds(North_China_Plain)\
                  .filterDate(year_start, year_end)\
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
                  .map(lambda x: maskS2clouds(x))\
                  .median()


# import Fourier img   
Fourier_img = ee.Image(f"users/wang8052664/North_China_Plain/Fourier_imgs/Fourier_img_{year_name}_harmonic_3")

# Import the Index-mean Img
NDVI_img = ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year_name}_Mean_NDVI").rename('Mean_NDVI')
NDBI_img = ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year_name}_Mean_NDBI").rename('Mean_NDBI')
EVI_img  = ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year_name}_Mean_EVI").rename('Mean_EVI') 

# Import DEM/SLOPE Img
DEM   = ee.Image("USGS/SRTMGL1_003").rename('DEM')
SLOPE = ee.Terrain.slope(DEM).rename('SLOPE')

In [10]:
# stack all Imput_Img together
Stack_img = ee.Image([Sentinel_img, Fourier_img, NDVI_img, NDBI_img, EVI_img, DEM, SLOPE, Climate_img])

##### Prepare input sample points

In [16]:
# Get the training sample
Control_points = ee.FeatureCollection('users/wangjinzhulala/North_China_Plain_Python/Sample_Points/Control_points')

In [66]:
rg = re.compile('^B')
mt = filter(rg.match,input_band)

In [67]:
sent_bands = list(mt)


In [68]:
sent_bands

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']

In [17]:
# define the gee-asset path for exporting
path   = 'users/wangjinzhulala/North_China_Plain_Python/Sample_extract_img'
name   = 'Sentinel_sample_ext_img'
sample = Control_points

Ext_sample = Stack_img.sampleRegions(  collection  = sample, 
                                        properties = ['Built'], 
                                        # here need to change scale to 10. If set as 30, the GEE will have to 
                                        # resample Sentinel img, which will leed to a 'Memory exceed problem' 
                                        scale      = 10, 
                                        geometries = True)

# exporting
task = ee.batch.Export.table.toAsset(   collection  = Ext_sample,
                                        description = f'{name}_2019',
                                        assetId     = f'{path}/{name}_2019')
task.start()


# print out the process
print(f'{name}_2019')

Sentinel_sample_ext_img_2019


In [20]:
# Define the path to Sample_pt
path = 'users/wangjinzhulala/North_China_Plain_Python/Sample_extract_img'

# Get the training sample
Training_sample = ee.FeatureCollection(f"{path}/Sentinel_sample_ext_img_2019")

#### Create the classification instances

In [69]:
# A two-layer loop to generate 10-randome-split layers for each year-period

Classificatioin_result = []



classificaiton_tpye = 'Sentinel'
year                = year_name
input_img           = Stack_img
sample_pt           = Training_sample
input_band          = input_img.bandNames().getInfo()

# The second-layer to go through each-randomnes
for seed in range(10):

    # Instatiate the class with a name.
    classification = Classification(year_name      = year,
                                    Verified_point = sample_pt,
                                    Input_img      = input_img,
                                    Input_band     = sent_bands,
                                    Tree_num       = 100,
                                    seed           = seed,
                                    Zone_sample    = None,
                                    classProperty  = 'Built')


    # perform the classification on Input_img and Samples
    classification.Stp_1_Classification_on_img()
    classification.Stp_2_Classification_on_Samples()

    # Get the classified img
    classified_img = classification.classification_img

    # Get the classified samples
    train_sample_classified = classification.Train_sample_classification
    test_sample_classified  = classification.Test_sample_classification

    # add classfication into the list
    Classificatioin_result.append((classificaiton_tpye,year,seed,classified_img,test_sample_classified))

In [70]:
# convert classificaiton instances into a dataframe
Classification_instances_df =  pd.DataFrame(Classificatioin_result,
                                            columns=['classificaiton_tpye','year','seed','classified_img','test_sample_classified'])
Classification_instances_df

,classificaiton_tpye,year,seed,classified_img,test_sample_classified
0,Sentinel,2017_2019,0,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
1,Sentinel,2017_2019,1,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
2,Sentinel,2017_2019,2,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
3,Sentinel,2017_2019,3,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
4,Sentinel,2017_2019,4,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
5,Sentinel,2017_2019,5,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
6,Sentinel,2017_2019,6,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
7,Sentinel,2017_2019,7,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
8,Sentinel,2017_2019,8,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
9,Sentinel,2017_2019,9,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."


#### Add one of the classified img to map

In [71]:
one = Classification_instances_df.iloc[0][3]
one

In [72]:
Full_sample_classified_img = ee.ImageCollection("users/wangjinzhulala/North_China_Plain_Python/classification_img/Random_2017_2019")\
                               .sum()

In [73]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.addLayer(one,{'min':0,'max':1},'One')
Map.addLayer(Full_sample_classified_img,{'min':0,'max':1},'Full_sample_classified_img')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

#### calculate the accuracy of each radomness

In [74]:
acc = []

# calculate the accuracy of differnet randomness
for i in Classificatioin_result:
    
    accuracy = Accuracy_assesment(i[4]).Stp_1_Calculate_Accuracy()
    acc.append(accuracy)
    
    key = 'Recall_built'
    print(f'{i[1]}_{i[2]}-->Recall_built:{accuracy[key]}')

2017_2019_0-->Recall_built:89.0
2017_2019_1-->Recall_built:88.46
2017_2019_2-->Recall_built:89.73
2017_2019_3-->Recall_built:87.63
2017_2019_4-->Recall_built:87.29
2017_2019_5-->Recall_built:88.35
2017_2019_6-->Recall_built:88.83
2017_2019_7-->Recall_built:86.99
2017_2019_8-->Recall_built:87.17
2017_2019_9-->Recall_built:88.82


##### Export the classified_img

In [75]:
Classification_instances_df

,classificaiton_tpye,year,seed,classified_img,test_sample_classified
0,Sentinel,2017_2019,0,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
1,Sentinel,2017_2019,1,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
2,Sentinel,2017_2019,2,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
3,Sentinel,2017_2019,3,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
4,Sentinel,2017_2019,4,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
5,Sentinel,2017_2019,5,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
6,Sentinel,2017_2019,6,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
7,Sentinel,2017_2019,7,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
8,Sentinel,2017_2019,8,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
9,Sentinel,2017_2019,9,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."


In [77]:
for item in Classificatioin_result:
     
    Classification_type = item[0]
    year = item[1]
    seed = item[2]
    img  = item[3]
    
    
    export_name = f'{Classification_type}_{year}_{seed}'
    
#     # export to Google Storage
#     task = ee.batch.Export.image.toCloudStorage(image          = img,
#                                                 description    = export_name,
#                                                 bucket         = 'north_china_plain',
#                                                 fileNamePrefix = export_name,
#                                                 region         = North_China_Plain.geometry().bounds(),
#                                                 scale          = 30,
#                                                 maxPixels      = int(1e13),
#                                                 skipEmptyTiles = True)
    
    #export to GEE asset
    asset_path = f'users/wangjinzhulala/North_China_Plain_Python/classification_img/Sentinel_2019'
    
    task = ee.batch.Export.image.toAsset(   image          = img,
                                            description    = export_name,
                                            assetId        = f'{asset_path}/{export_name}',
                                            region         = North_China_Plain.geometry().bounds(),
                                            scale          = 30,
                                            maxPixels      = int(1e13)) 
    
    task.start()
    
    print(f'Exporting {export_name} successful!')

Exporting Sentinel_2017_2019_0 successful!
Exporting Sentinel_2017_2019_1 successful!
Exporting Sentinel_2017_2019_2 successful!
Exporting Sentinel_2017_2019_3 successful!
Exporting Sentinel_2017_2019_4 successful!
Exporting Sentinel_2017_2019_5 successful!
Exporting Sentinel_2017_2019_6 successful!
Exporting Sentinel_2017_2019_7 successful!
Exporting Sentinel_2017_2019_8 successful!
Exporting Sentinel_2017_2019_9 successful!
